### Chatbot History implemntation with sesson id details

In [1]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")


/home/maniram/workspace/python/langchain-projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="groq/compound",groq_api_key=groq_api_key)
model

ChatGroq(profile={}, client=<groq.resources.chat.completions.Completions object at 0x7f5b69847b00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f5b69744620>, model_name='groq/compound', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="hi i am maniram and i am software engineer")])

AIMessage(content='Hi Maniram! Great to meet a fellow software engineer. How can I help you today?', additional_kwargs={'reasoning_content': '<Think>\n\n</Think>'}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 250, 'total_tokens': 315, 'completion_time': 0.141887, 'completion_tokens_details': None, 'prompt_time': 0.008647, 'prompt_tokens_details': None, 'queue_time': 0.108736, 'total_time': 0.150534}, 'model_name': 'groq/compound', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b987c-9d22-7a33-8acd-f2f28643af14-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 250, 'output_tokens': 65, 'total_tokens': 315})

In [4]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="hi i am maniram and i am software engineer"),
    AIMessage(content="Hello maniram, nice to meet you, how can I help you ?"),
    HumanMessage(content="what's my name and who I am ?"),
    
])

AIMessage(content='Your name is **Maniram**, and you mentioned that you’re a **software engineer**.', additional_kwargs={'reasoning_content': '<Think>\n\n</Think>'}, response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 318, 'total_tokens': 415, 'completion_time': 0.21274, 'completion_tokens_details': None, 'prompt_time': 0.011271, 'prompt_tokens_details': None, 'queue_time': 0.128751, 'total_time': 0.224012}, 'model_name': 'groq/compound', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b987c-9f02-7730-949d-bb8317c7147f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 318, 'output_tokens': 97, 'total_tokens': 415})

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str) -> BaseChatMessageHistory :
    if session_id not in store :
        store[session_id]=ChatMessageHistory()
    return store[session_id]
with_message_history=RunnableWithMessageHistory(model,get_session_history)


In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , my name is Maniram an I am AI Engineer")],
    config
)
response.content

'Hello Maniram! Nice to meet you. How can I assist you today?'

In [8]:
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="What is my name?" )],
    config1
)
response.content

'I’m sorry, but I don’t have any information about your name. In our earlier exchange I noted that, based on the context available to me, there’s no data that would let me determine or infer your name. Since I have no record of you providing it, I can’t answer the question. If you’d like me to address you by a particular name, just let me know!'

# PromptTemplate

In [9]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate(
    [
        ("system","you are a helpful assistant. Answer all questions to the best of your ability.",),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [10]:
chain.invoke({"messages":[HumanMessage(content="My name is Krish")]})

AIMessage(content='Nice to meet you, Krish! How can I help you today?', additional_kwargs={'reasoning_content': '<Think>\n\n</Think>'}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 278, 'total_tokens': 355, 'completion_time': 0.169168, 'completion_tokens_details': None, 'prompt_time': 0.011312, 'prompt_tokens_details': None, 'queue_time': 0.104887, 'total_time': 0.180479}, 'model_name': 'groq/compound', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b987c-b251-7b51-b529-8357501c2887-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 278, 'output_tokens': 77, 'total_tokens': 355})

In [11]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [12]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke([
    HumanMessage(content="Hi, My name is Krish")
],config=config)
response.content


'Hi Krish! Nice to meet you. How can I help you today?'

In [13]:
prompt=ChatPromptTemplate(
    [
        ("system","you are a helpful assistant. Answer all questions to the best of your ability in {language}.",),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [14]:
response=chain.invoke(
    {"messages":[HumanMessage(
        content="Hi My name is Krish"
    )],"language":"hindi"}
)
response.content

'**विचार प्रक्रिया (Reasoning):**  \n1. आपका पहला संदेश केवल एक अभिवादन और अपना नाम बताता था – “Hi My name is Krish”.  \n2. इस प्रकार के संदेश का मुख्य उद्देश्य परिचय देना और संवाद की शुरुआत करना होता है।  \n3. इसलिए सबसे उपयुक्त प्रतिक्रिया एक मित्रवत अभिवादन के साथ आपका स्वागत करना और यह पूछना है कि मैं आपकी किस प्रकार मदद कर सकता/सकती हूँ।  \n\n**हिंदी में उत्तर:**  \n\nनमस्ते\u202fकृष! आपका स्वागत है। मैं आपकी किस प्रकार मदद कर सकता/सकती हूँ? यदि कोई प्रश्न है या किसी जानकारी की ज़रूरत है, तो बेझिझक बताइए।'

In [15]:
with_message_history=RunnableWithMessageHistory(
    chain,get_session_history,
    input_messages_key="messages"
)


In [16]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi,my name is Krish")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते कृष! आपका स्वागत है। मैं आपकी कैसे मदद कर सकता हूँ?'

## Messaging with Conversation History

In [17]:
from langchain_core.messages import SystemMessage
from langchain_core.messages.utils import trim_messages
trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages=[
    SystemMessage(content="you ar e a good assistant"),
    HumanMessage(content="Hi, I am bob"),
    AIMessage(content="hi"),
    HumanMessage(content="I like vanilla icecream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),

]

trimmer.invoke(messages)

[SystemMessage(content='you ar e a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, I am bob', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='I like vanilla icecream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='whats 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs=

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain = (RunnablePassthrough.assign(
    messages=itemgetter("messages")|trimmer
    ) | prompt
    | model)

response=chain.invoke(
   { "messages": messages + [ HumanMessage(content="What ice cream do i like") ],
    "language":"English"
   }
)
response.content

'The problem you asked was the math question **“What is 2\u202f+\u202f4?”** (i.e., “whats 2+2”).'

In [27]:
response=chain.invoke(
   { "messages": messages + [ HumanMessage(content="What problem did i ask") ],
    "language":"English"
   }
)
response.content

'The only actual problem you asked was the math question:\n\n**“whats 2+2”**'

In [30]:
response = with_message_history.invoke(
    {
        "messages":messages + [ HumanMessage(content="Whats my name ?")],
        "language":"English",
    },
    config=config
)
response.content

'**Answer:**\u202fYour name is **Bob**.\n\n**Reasoning:**\n\n1. In the very first message you said, “Hi, my name is **Krish**.”\n2. A few messages later you wrote, “Hi, I am **bob**.”\n3. When a user provides more than one name, the most recent self‑identification is generally taken as the current name they wish to be called.\n4. Therefore, based on the latest statement, the name you are using now is **Bob**.'